In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from datetime import datetime

In [2]:
import os
import geopandas as gpd

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

USER = os.environ["USER"]
PASSWORD = os.environ["PASSWORD"]
DOMAIN = os.getenv('DOMAIN')

In [4]:
#API access to get bike stations location

url = "https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/"

headers = {
    'email': USER + "@" + DOMAIN,
    'password': PASSWORD 
    }

response = requests.request("GET", url, headers=headers)
#print(response.text)

valid_token = json.loads(response.text)["data"][0]["accessToken"]

In [5]:
stations_url = "https://openapi.emtmadrid.es/v1/transport/bicimad/stations/"

headers = {
    'accesstoken': valid_token

    }

response = requests.request("GET", stations_url, headers=headers)

stations_json_raw = response.json()


In [6]:
df_raw = pd.DataFrame(stations_json_raw['data'])
df_raw['id_'] = df_raw.index

In [7]:
# The dataset with the stations information
df_raw.head()

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry,id_
0,1,Puerta del Sol A,2,1a,Puerta del Sol nº 1,1,0,30,12,18,0,"{'type': 'Point', 'coordinates': [-3.7018341, ...",0
1,2,Puerta del Sol B,2,1b,Puerta del Sol nº 1,1,0,30,17,13,0,"{'type': 'Point', 'coordinates': [-3.701602938...",1
2,3,Miguel Moya,0,2,Calle Miguel Moya nº 1,1,0,24,4,19,0,"{'type': 'Point', 'coordinates': [-3.7058415, ...",2
3,4,Plaza Conde Suchil,0,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,4,12,0,"{'type': 'Point', 'coordinates': [-3.7069171, ...",3
4,5,Malasaña,0,4,Calle Manuela Malasaña nº 5,1,0,24,5,18,0,"{'type': 'Point', 'coordinates': [-3.7025875, ...",4


In [8]:
df_raw["geometry"][0] # latitud longitud

{'type': 'Point', 'coordinates': [-3.7018341, 40.4172137]}

In [9]:
df_geo = pd.DataFrame(df_raw["geometry"].values.tolist(), index=df_raw.index)
df_geo.head(2)

,type,coordinates
0,Point,"[-3.7018341, 40.4172137]"
1,Point,"[-3.701602938060457, 40.41731271011562]"


In [22]:
df_geo[['Longitude', 'Latitude']] = pd.DataFrame(df_geo.coordinates.tolist(), index= df_geo.index)
df_geo["id_"] = df_geo.index
df_geo.head(2)

,type,coordinates,Longitude,Latitude,id_
0,Point,"[-3.7018341, 40.4172137]",-3.701834,40.417214,0
1,Point,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313,1


In [23]:
gdf = gpd.GeoDataFrame(df_raw, geometry=gpd.points_from_xy(df_geo.Longitude, df_geo.Latitude))
gdf.head(2)

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry,id_
0,1,Puerta del Sol A,2,1a,Puerta del Sol nº 1,1,0,30,12,18,0,POINT (-3.70183 40.41721),0
1,2,Puerta del Sol B,2,1b,Puerta del Sol nº 1,1,0,30,17,13,0,POINT (-3.70160 40.41731),1


In [24]:
bike_station_df = pd.merge(df_raw, df_geo, on='id_')

In [25]:
bike_station_df.head()

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry,id_,type,coordinates,Longitude,Latitude
0,1,Puerta del Sol A,2,1a,Puerta del Sol nº 1,1,0,30,12,18,0,POINT (-3.70183 40.41721),0,Point,"[-3.7018341, 40.4172137]",-3.701834,40.417214
1,2,Puerta del Sol B,2,1b,Puerta del Sol nº 1,1,0,30,17,13,0,POINT (-3.70160 40.41731),1,Point,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313
2,3,Miguel Moya,0,2,Calle Miguel Moya nº 1,1,0,24,4,19,0,POINT (-3.70584 40.42059),2,Point,"[-3.7058415, 40.4205886]",-3.705842,40.420589
3,4,Plaza Conde Suchil,0,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,4,12,0,POINT (-3.70692 40.43029),3,Point,"[-3.7069171, 40.4302937]",-3.706917,40.430294
4,5,Malasaña,0,4,Calle Manuela Malasaña nº 5,1,0,24,5,18,0,POINT (-3.70259 40.42855),4,Point,"[-3.7025875, 40.4285524]",-3.702587,40.428552


In [26]:
air_q_stations = pd.read_csv('../data/informacion_estaciones_red_calidad_aire.csv', error_bad_lines=False, sep = ';', encoding = "ISO-8859-1")
air_q_stations.head(2)

,CODIGO,CODIGO_CORTO,ESTACION,DIRECCION,LONGITUD_ETRS89,LATITUD_ETRS89,ALTITUD,COD_TIPO,NOM_TIPO,NO2,...,HC,COD_VIA,VIA_CLASE,VIA_PAR,VIA_NOMBRE,Fecha alta,COORDENADA_X_ETRS89,COORDENADA_Y_ETRS89,LONGITUD,LATITUD
0,28079004,4,Pza. de España,Plaza de España,"3°42'43.91""O","40°25'25.98""N",637,UT,Urbana tráfico,X,...,NaN,273600,PLAZA,DE,ESPAÑA,01/12/1998,"439579,3291","4475049,263",-3.712257,40.423882
1,28079008,8,Escuelas Aguirre,Entre C/ Alcalá y C/ O Donell,"3°40'56.22""O","40°25'17.63""N",672,UT,Urbana tráfico,X,...,X,18900,CALLE,DE,ALCALA,01/12/1998,"442117,2366","4474770,696",-3.682316,40.421553


In [27]:
location = [40.41955949449261,-3.6888147164886242]
air_Map = folium.Map(
                location = location, 
                zoom_start = 14)

latitudes = list(air_q_stations.LONGITUD)
longitudes = list(air_q_stations.LATITUD)
labels = list(air_q_stations.ESTACION)
for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker(
      location = [lng, lat], 
      popup = label,
      icon = folium.Icon()
     ).add_to(air_Map)
air_Map

In [28]:
#hacer geopoints y hacer listado de puntos a max 15 m de distancia
air_stations_df = gpd.GeoDataFrame(air_q_stations, geometry=gpd.points_from_xy(air_q_stations.LATITUD, air_q_stations.LONGITUD))

In [29]:
air_stations_df.columns

Index(['CODIGO', 'CODIGO_CORTO', 'ESTACION', 'DIRECCION', 'LONGITUD_ETRS89',
       'LATITUD_ETRS89', 'ALTITUD', 'COD_TIPO', 'NOM_TIPO', 'NO2', 'SO2', 'CO',
       'PM10', 'PM2_5', 'O3', 'BTX', 'HC', 'COD_VIA', 'VIA_CLASE', 'VIA_PAR',
       'VIA_NOMBRE', 'Fecha alta', 'COORDENADA_X_ETRS89',
       'COORDENADA_Y_ETRS89', 'LONGITUD', 'LATITUD', 'geometry'],
      dtype='object')

In [30]:
def min_distance(bici_pts, air_pts):
    return bici_pts.distance(air_pts).min()

In [31]:
bikes_geolocation = bike_station_df[["id", "name", "address", "total_bases", "geometry", "Latitude", "Longitude"]]
display(bikes_geolocation.head())

,id,name,address,total_bases,geometry,Latitude,Longitude
0,1,Puerta del Sol A,Puerta del Sol nº 1,30,POINT (-3.70183 40.41721),40.417214,-3.701834
1,2,Puerta del Sol B,Puerta del Sol nº 1,30,POINT (-3.70160 40.41731),40.417313,-3.701603
2,3,Miguel Moya,Calle Miguel Moya nº 1,24,POINT (-3.70584 40.42059),40.420589,-3.705842
3,4,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,18,POINT (-3.70692 40.43029),40.430294,-3.706917
4,5,Malasaña,Calle Manuela Malasaña nº 5,24,POINT (-3.70259 40.42855),40.428552,-3.702587


In [33]:
air_geolocation = air_stations_df[["CODIGO", "ESTACION", "LONGITUD", "LATITUD","geometry"]]
air_geolocation.head()

,CODIGO,ESTACION,LONGITUD,LATITUD,geometry
0,28079004,Pza. de España,-3.712257,40.423882,POINT (40.42388 -3.71226)
1,28079008,Escuelas Aguirre,-3.682316,40.421553,POINT (40.42155 -3.68232)
2,28079011,Avda. Ramón y Cajal,-3.677349,40.451473,POINT (40.45147 -3.67735)
3,28079016,Arturo Soria,-3.639242,40.440046,POINT (40.44005 -3.63924)
4,28079017,Villaverde,-3.713317,40.347147,POINT (40.34715 -3.71332)


In [36]:
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0

lat1 = radians(52.2296756)
lon1 = radians(21.0122287)
lat2 = radians(52.406374)
lon2 = radians(16.9251681)

dlon = lon2 - lon1
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
print("Should be:", 278.546, "km")

def _distance(lat1, lon1, lat2, lon2):
    R = 6373.0 # approximate radius of earth in km

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c * 1000
    return distance

Result: 278.54558935106695
Should be: 278.546 km


In [48]:
# bikes_geolocation air_geolocation
# for each bike station, which air station it's closer
lat_bike = bikes_geolocation.Latitude.loc[0]
lon_bike = bikes_geolocation.Longitude.loc[0]
lista_ = []
for code, lat, lon in zip(list(air_geolocation.CODIGO), list(air_geolocation.LATITUD), list(air_geolocation.LONGITUD)):
    lista_.append((code, _distance(lat_bike,lon_bike, lat, lon)))
    
min(lista_, key = lambda t: t[1])

(28079035, 248.97012113475435)

In [51]:
all_distances = [(code, _distance(lat_bike,lon_bike, lat, lon)) for code, lat, lon in zip(list(air_geolocation.CODIGO), list(air_geolocation.LATITUD), list(air_geolocation.LONGITUD))]
min_station_distance = min(all_distances, key = lambda t: t[1])
station_code = min_station_distance[0]
distance = min_station_distance[0]

In [59]:
# bikes_geolocation air_geolocation
# ["id", "name", "address", "total_bases", "geometry", "Latitude", "Longitude"]
b_id = list(bikes_geolocation.id)
b_lat = list(bikes_geolocation.Latitude)
b_lon = list(bikes_geolocation.Longitude)

def min_dist_airStation(x):
    air_geolocation_ = air_geolocation
    all_distances = [(code, _distance(x.Latitude,x.Longitude, lat, lon)) for code, lat, lon in zip(list(air_geolocation_.CODIGO), list(air_geolocation_.LATITUD), list(air_geolocation_.LONGITUD))]
    min_station_distance = min(all_distances, key = lambda t: t[1])
    station_code = min_station_distance[0]
    distance = min_station_distance[0]
    return station_code, distance

bikes_geolocation[["air_station_code", "min_dist"]] = bikes_geolocation[["Latitude", "Longitude"]].apply(min_dist_airStation) 

AttributeError: 'Series' object has no attribute 'Latitude'

In [56]:
bikes_geolocation

,id,name,address,total_bases,geometry,Latitude,Longitude,id_
0,1,Puerta del Sol A,Puerta del Sol nº 1,30,POINT (-3.70183 40.41721),40.417214,-3.701834,41.417214
1,2,Puerta del Sol B,Puerta del Sol nº 1,30,POINT (-3.70160 40.41731),40.417313,-3.701603,42.417313
2,3,Miguel Moya,Calle Miguel Moya nº 1,24,POINT (-3.70584 40.42059),40.420589,-3.705842,43.420589
3,4,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,18,POINT (-3.70692 40.43029),40.430294,-3.706917,44.430294
4,5,Malasaña,Calle Manuela Malasaña nº 5,24,POINT (-3.70259 40.42855),40.428552,-3.702587,45.428552
...,...,...,...,...,...,...,...,...
259,265,INEF,Avenida Juan de Herrera frente a la calle Paul...,24,POINT (-3.72997 40.43896),40.438960,-3.729970,305.438960
260,266,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,24,POINT (-3.72699 40.44375),40.443750,-3.726990,306.443750
261,267,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,24,POINT (-3.72693 40.44342),40.443420,-3.726930,307.443420
262,268,Facultad Biología,Calle José Antonio Novais frente al nº 12,24,POINT (-3.72731 40.44912),40.449120,-3.727310,308.449120


In [44]:
import numpy as np
# add columns with radians for latitude and longitude
df_pts_bikes[['lat_radians_','long_radians_']] = (
    np.radians(df_pts_bikes.loc[:,['Latitude','Longitude']])
)
df_pts_air[['lat_radians_','long_radians_']] = (
    np.radians(df_pts_air.loc[:,['LATITUD','LONGITUD']])
)

/home/fserrey/anaconda3/envs/jupyter_env/lib/python3.7/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/fserrey/anaconda3/envs/jupyter_env/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [46]:
import sklearn.neighbors

In [63]:
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')

dist_matrix = (dist.pairwise
    (df_pts_air[['lat_radians_','long_radians_']],
     df_pts_bikes[['lat_radians_','long_radians_']])#*6371000
)
dist_matrix

array([[1.04325768, 1.04325555, 1.04334898, ..., 1.04388359, 1.04394974,
        1.04399918],
       [1.04291366, 1.04291153, 1.04300495, ..., 1.04353954, 1.04360567,
        1.04365511],
       [1.04317129, 1.04316916, 1.04326259, ..., 1.04379713, 1.04386322,
        1.04391266],
       ...,
       [1.0449005 , 1.04489837, 1.04499179, ..., 1.04552632, 1.04559239,
        1.04564183],
       [1.04261655, 1.04261441, 1.04270784, ..., 1.04324232, 1.04330835,
        1.04335779],
       [1.04381313, 1.043811  , 1.04390442, ..., 1.0444389 , 1.04450494,
        1.04455438]])

In [66]:
# Note that 6378 is the radius of the earth in km
df_dist_matrix = (
    pd.DataFrame(dist_matrix,
                 index = df_pts_air['CODIGO'], 
                 columns = df_pts_bikes['id'])  
)
df_dist_matrix.head()

id,1,2,3,4,5,6,7,8,9,10,...,260,261,262,263,264,265,266,267,268,269
CODIGO,,,,,,,,,,,,,,,,,,,,,
28079004,1.043258,1.043256,1.043349,1.043468,1.043389,1.043381,1.043285,1.043287,1.043283,1.043346,...,1.043592,1.043482,1.043379,1.043524,1.043734,1.043878,1.043888,1.043884,1.043950,1.043999
28079008,1.042914,1.042912,1.043005,1.043123,1.043045,1.043037,1.042941,1.042943,1.042939,1.043002,...,1.043248,1.043138,1.043035,1.043179,1.043390,1.043534,1.043544,1.043540,1.043606,1.043655
28079011,1.043171,1.043169,1.043263,1.043381,1.043303,1.043295,1.043199,1.043200,1.043197,1.043260,...,1.043505,1.043395,1.043292,1.043437,1.043648,1.043791,1.043801,1.043797,1.043863,1.043913
28079016,1.042646,1.042644,1.042737,1.042856,1.042777,1.042770,1.042673,1.042675,1.042671,1.042735,...,1.042979,1.042869,1.042766,1.042911,1.043122,1.043266,1.043276,1.043272,1.043338,1.043387
28079017,1.042473,1.042471,1.042565,1.042683,1.042605,1.042597,1.042501,1.042503,1.042499,1.042562,...,1.042809,1.042698,1.042595,1.042740,1.042950,1.043093,1.043104,1.043099,1.043166,1.043215


In [73]:
aux_df = pd.DataFrame(df_dist_matrix.idxmin(axis = 0, skipna = True), columns = ["station"])
                                                                                                 

,station
id,
1,28079054
2,28079054
3,28079054
4,28079054
5,28079054
...,...
264,28079054
265,28079054
266,28079054


In [65]:
# Unpivot this dataframe from wide format to long format.
# When you unpivot, the data in the pivot table becomes a
# column named 'value'. Rename this column to 'miles' for clarity.
df_dist_long = (
    pd.melt(df_dist_matrix.reset_index(),id_vars='id')
)
df_dist_long = df_dist_long.rename(columns={'value':'m'})

df_dist_long.shape

KeyError: 'id'

In [26]:
bici_points["min_distance_to_stations"] = bici_points.geometry.apply(min_distance, air_points["geometry"])

ValueError: The truth value of a GeoSeries is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [18]:
#from geopy.distance import distance 

coor_bici_stations_closer = [bici_points[['name','geometry']] for i,a in zip(list(air_points['geometry']), list(bici_points['geometry'])) if i.distance(a) < 15] 

In [19]:
coor_bici_stations_closer

[                       name                   geometry
 0          Puerta del Sol A  POINT (40.41721 -3.70183)
 1          Puerta del Sol B  POINT (40.41731 -3.70160)
 2               Miguel Moya  POINT (40.42059 -3.70584)
 3        Plaza Conde Suchil  POINT (40.43029 -3.70692)
 4                  Malasaña  POINT (40.42855 -3.70259)
 ..                      ...                        ...
 259                    INEF  POINT (40.43896 -3.72997)
 260  Ciudad Universitaria 1  POINT (40.44375 -3.72699)
 261  Ciudad Universitaria 2  POINT (40.44342 -3.72693)
 262       Facultad Biología  POINT (40.44912 -3.72731)
 263        Facultad Derecho  POINT (40.45109 -3.72937)
 
 [264 rows x 2 columns],
                        name                   geometry
 0          Puerta del Sol A  POINT (40.41721 -3.70183)
 1          Puerta del Sol B  POINT (40.41731 -3.70160)
 2               Miguel Moya  POINT (40.42059 -3.70584)
 3        Plaza Conde Suchil  POINT (40.43029 -3.70692)
 4                  M

In [20]:
from shapely.ops import nearest_points

In [21]:
nearest_points(bici_points["geometry"][0], pts3)


NameError: name 'pts3' is not defined

In [ ]:
pts3 = air_points.geometry.unary_union
def near(point, pts=pts3):
     # find the nearest point and return the corresponding Place value
    nearest = air_points.geometry == nearest_points(point, pts)[1]
    return air_points.CODIGO[0]

bici_points['CODIGO'] = bici_points["geometry"].apply(lambda x: near(x))
bici_points.head()

In [ ]:
df = pd.merge(bici_points, air_points, on='CODIGO', how = 'left')

In [ ]:
df_aux = df[['id','name', 'geometry_x', "CODIGO", "ESTACION", "geometry_y"]]
df_aux.head()

In [ ]:
def getXY(pt):
    return (pt.x, pt.y)

centroidseries = df_aux['geometry_x'].apply(lambda x: x.centroid)
df_aux['bici_lon'],df_aux['bici_lat'] = [list(t) for t in zip(*map(getXY, centroidseries))]

centroidseries = df_aux['geometry_y'].apply(lambda x: x.centroid)
df_aux['air_lon'],df_aux['air_lat'] = [list(t) for t in zip(*map(getXY, centroidseries))]

In [ ]:
location = [40.41955949449261,-3.6888147164886242]
mapa = folium.Map(
                location = location, 
                zoom_start = 14)

lon_bici = list(df_aux.bici_lon)
lat_bici = list(df_aux.bici_lat)
lon_air = list(df_aux.air_lon)
lat_air = list(df_aux.air_lat)

bici_stations_tag = list(df_aux.name)
air_stations_tag = list(df_aux.ESTACION)

for lat, lng, label in zip(lon_bici, lat_bici, bici_stations_tag):
    folium.Marker(
          location = [lat, lng], 
          popup = bici_stations_tag,
          icon = folium.Icon(color='blue')
         ).add_to(mapa) 
    
for lat, lng, label in zip(lon_air, lat_air, air_stations_tag):        
    folium.Marker(
          location = [lat, lng], 
          popup = air_stations_tag,
          icon = folium.Icon(color='green')
         ).add_to(mapa)
mapa

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

lat_point_list = [50.854457, 48.853033, 52.518172, 50.072651, 50.854457]
lon_point_list = [4.377184, 2.349553, 13.407759, 14.435935, 4.377184]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
polygon_geom2 = polygon_geom.convex_hull # the atribute
import folium
m = folium.Map([50.854457, 4.377184], zoom_start=5, tiles='cartodbpositron')
folium.GeoJson(polygon_geom).add_to(m)
folium.GeoJson(polygon_geom2).add_to(m)
folium.LatLngPopup().add_to(m)
m

In [ ]:
polygon_geom

In [ ]:
"""
para I en lista de bicis
    mide la distancia con cada estación
    si la distacia es < 15 m
        entonces hazme un listado de códigos
"""

In [ ]:
def getXY(pt):
    return (pt.x, pt.y)

centroidseries = bici_points['geometry'].apply(lambda x: x.centroid)
bici_points['bici_lon'],bici_points['bici_lat'] = [list(t) for t in zip(*map(getXY, centroidseries))]

centroidseries = air_points['geometry'].apply(lambda x: x.centroid)
air_points['air_lon'],air_points['air_lat'] = [list(t) for t in zip(*map(getXY, centroidseries))]

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r * 1000


In [ ]:
bici_lon = list(bici_points['bici_lon'])
bici_lat = list(bici_points['bici_lat'])

air_lon = list(air_points['air_lon'])
air_lat = list(air_points['air_lat'])

air_code = list(air_points['CODIGO'])


lista = []
coor = []
for i,j in zip(bici_lon, bici_lat):
    l  = []
    coor_ = []
    for y,z in zip(air_lon, air_lat):
        if haversine(i,j,y,z) < 15:
            l.append(haversine(i,j,y,z))
            coor_.append((y,z))
    lista.append(l)
    coor.append(coor_)

In [ ]:
tup_coor = [set(i) for i in coor]

len(tup_coor)

In [ ]:
for i in tup_coor:
    if i == (air_points['air_lon'], air_points['air_lat'])
    

In [ ]:
df_ = bicis_points[['id','name', 'geometry_x','air_lon','air_lat']]
df_["dist"] = lista
df_

"""
indices = [i for i, x in enumerate(my_list) if x == "whatever"]

si estas coordenadas están en la lista, dime el codigo de la misma fila
"""